In [1]:
import requests
import pandas as pd
import numpy as np
from io import StringIO
import re
import json
from tqdm import tqdm_notebook as tqdm
from bs4 import BeautifulSoup

In [2]:
def get_all_tournaments(idteam):
    url_team = 'https://rating.chgk.info/api/teams/{}/tournaments/'
    response = requests.get(url_team.format(idteam)).text
    response = json.load(StringIO(response))
    result = []
    for i in response:
        res = response[i]
        for j in res['tournaments']:
            result.append(j)
    return result

In [3]:
def get_editors(idtournament):
    url_tournament = 'https://rating.chgk.info/tournament/{}'
    response = requests.get(url_tournament.format(idtournament)).text
    soup = BeautifulSoup (response, 'html.parser')
    child_soup = soup.findAll(text=re.compile('Редакторы'))
    cards = []
    for i in child_soup:
        cards.append(i.parent.parent)
    editors = []
    for i in cards:
        editors.append(i.find_all('a', href=re.compile('player')))                   
    result = []
    for i in editors:
        for j in i:
            result.append(j.text)
    return result 

In [4]:
def get_bonus(idteam, idtournament):
    url_tm = 'https://rating.chgk.info/api/tournaments/{}/list'
    response = requests.get(url_tm.format(idtournament)).text
    response = json.load(StringIO(response))
    for i in response:
        if i['idteam'] == str(idteam):
            bonus = i['diff_bonus']
            break
    return int(bonus)

In [5]:
def get_all_editors_and_bonus(idteam):
    all_games = get_all_tournaments(idteam)
    all_editors = [get_editors(i) for i in tqdm(all_games)]    
    all_bonuses = [get_bonus(idteam, i) for i in tqdm(all_games)]
    d = {'games_id':all_games,'editors':all_editors, 'bonus':all_bonuses}
    df = pd.DataFrame(d)
    df = df[df['editors'].astype(str) != '[]']
    return df

In [6]:
eds_and_bons = get_all_editors_and_bonus(4109)
eds_and_bons

,games_id,editors,bonus
0,6032,"[Ерёмин Иван Андреевич, Терентьев Сергей Алекс...",-38
1,7087,"[Кунилов Андрей Александрович, Парр Марина, Де...",301
2,7188,"[Голуб Эдуард Викторович, Уланов Алексей Стани...",-15
3,7144,"[Саксонов Антон Владимирович, Рождествин Алекс...",58
4,7172,"[Алиев Руслан Рашидович, Савченков Михаил Влад...",248
...,...,...,...
545,2567,[Малкин Михаил Леонидович],-52
546,2526,"[Башук Дмитрий Николаевич, Коробейников Алекса...",-21
547,2472,[Моносов Борис Яковлевич],3
548,2479,"[Солахян Павел Викторович, Стрильчук Дмитрий Н...",-27


Считаем долю каждого редактора в командном бонусе

In [7]:
eds_and_bons['num_of_editors'] = eds_and_bons.apply(lambda row: len(row['editors']), axis=1)
eds_and_bons['bonus_share'] = eds_and_bons.apply(lambda row: row['bonus']/row['num_of_editors'], axis=1)
eds_and_bons

,games_id,editors,bonus,num_of_editors,bonus_share
0,6032,"[Ерёмин Иван Андреевич, Терентьев Сергей Алекс...",-38,3,-12.666667
1,7087,"[Кунилов Андрей Александрович, Парр Марина, Де...",301,9,33.444444
2,7188,"[Голуб Эдуард Викторович, Уланов Алексей Стани...",-15,3,-5.000000
3,7144,"[Саксонов Антон Владимирович, Рождествин Алекс...",58,5,11.600000
4,7172,"[Алиев Руслан Рашидович, Савченков Михаил Влад...",248,2,124.000000
...,...,...,...,...,...
545,2567,[Малкин Михаил Леонидович],-52,1,-52.000000
546,2526,"[Башук Дмитрий Николаевич, Коробейников Алекса...",-21,4,-5.250000
547,2472,[Моносов Борис Яковлевич],3,1,3.000000
548,2479,"[Солахян Павел Викторович, Стрильчук Дмитрий Н...",-27,5,-5.400000


Присваиваем каждому редактору на каждом турнире его долю в бонусе

In [8]:
eds_and_bons = eds_and_bons.explode('editors')
eds_and_bons

,games_id,editors,bonus,num_of_editors,bonus_share
0,6032,Ерёмин Иван Андреевич,-38,3,-12.666667
0,6032,Терентьев Сергей Александрович,-38,3,-12.666667
0,6032,Колмаков Игорь,-38,3,-12.666667
1,7087,Кунилов Андрей Александрович,301,9,33.444444
1,7087,Парр Марина,301,9,33.444444
...,...,...,...,...,...
548,2479,Шорин Сергей Александрович,-27,5,-5.400000
552,2370,Крапиль Николай Валерьевич,49,4,12.250000
552,2370,Малкин Михаил Леонидович,49,4,12.250000
552,2370,Сахаров Константин Владимирович,49,4,12.250000


Считаем сумму всех бонусов команды на конкретном редакторе

In [9]:
eds_sum_share = eds_and_bons.groupby('editors').agg({'bonus_share':'sum'}).sort_values(by='bonus_share', ascending=False)
eds_sum_share

,bonus_share
editors,
Голуб Эдуард Викторович,865.829143
Колесов Артём Александрович,786.100649
Терентьев Сергей Александрович,773.062897
Бороненко Алексей Владимирович,701.807937
Мерзляков Максим Петрович,697.161255
...,...
Кононенко Евгений Иванович,-125.000000
Цалкович Егор Александрович,-160.750000
Науменко Константин Владимирович,-176.750000


Считаем медианный бонус команды на конкретном редакторе

In [10]:
eds_median_share = eds_and_bons.groupby('editors').agg({'bonus_share':'median'}).sort_values(by='bonus_share', ascending=False)
eds_median_share

,bonus_share
editors,
Пятов Игорь Александрович,157.000000
Рубашкина Анастасия Дмитриевна,146.500000
Орловецкий Борис Константинович,146.500000
Гусаков Антон Вадимович,141.000000
Киселёв Иван Сергеевич,79.333333
...,...
Семёнов Артём Александрович,-51.666667
Калякин Юрий Алексеевич,-60.000000
Соловьёв Дмитрий Евгеньевич,-60.000000


In [11]:
eds_and_bons.to_csv('./eds_and_bons.csv', index=False)